<a href="https://colab.research.google.com/github/gabrielbribeiroo/OperationalResearch_UFPB/blob/main/Pratica_Destinacao_Produtos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula prática: Destinação de Produtos à Fabricas
<sup>Exemplo do capítulo 8.3 do livro `Introdução à Pesquisa Operacional` de `Hillier e Lieberman`.</sup>

## Exercício 1

### Descrição do problema
A Cia. Produtos Melhores decidiu iniciar a produção de quatro produtos novos usando três fábricas que, no momento, têm excesso de capacidade produtiva. Os produtos requerem um esforço de produção comparável por unidade, de modo que a capacidade produtiva disponível das fábricas seja medida pelo número de unidades de qualquer produto que possa ser produzido diariamente, conforme dado na tabela 1. A tabela 2 fornece a taxa de produção diária necessária para atender às vendas projetadas. Cada fábrica é capaz de produzir qualquer um desses produtos, exceto a Fábrica 2, que não pode fabricar o produto 3. Os custos variáveis por unidade de cada produto diferem de fábrica para fábrica, conforme a tabela 3.

Tabela 1: Capacidade de produção disponível, por unidade do produto

| | Fábrica 1 | Fábrica 2 | Fábrica 3 |
|:---|:---:|:---:|:---:|
| Capacidade disponível | 75 | 75 | 45 |

Tabela 2: Demanda de produção diária

| | Produto 1 | Produto 2 | Produto 3 | Produto 4 |
|:---|:---:|:---:|:---:|:---:|
| Demanda | 20 | 30 | 30 | 40 |

Tabela 3: Custo unitário por produto

| | Produto 1 | Produto 2 | Produto 3 | Produto 4 |
|:---|:---:|:---:|:---:|:---:|
| Fábrica 1 | 41 | 27 | 28 | 24 |
| Fábrica 2 | 40 | 29 | - | 23 |
| Fábrica 3 | 37 | 30 | 27 | 21 |

A gerência precisa tomar uma decisão sobre como dividir a fabricação dos produtos entre as fábricas, e decidiu permitir a divisão da produção de um mesmo produto em mais de uma fábrica.

Escreva um modelo para determinar quais fábricas produzirão quais produtos e a que custo total.

### Resolução

#### Instalação do Mip e funções usadas posteriormente

In [ ]:
!pip install mip
from mip import *

In [ ]:
# funcões usadas posteriormente:

# resolve o modelo e mostra os valores das variáveis
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  print(f"Solution value  = {model.objective_value:.2f}\n")

  print("Solution:")
  for v in model.vars:
      print(f"{v.name} = {v.x:.2f}")


# salva modelo em arquivo lp, e mostra o conteúdo
def save(model, filename):
  model.write(filename) # salva modelo em arquivo
  with open(filename, "r") as f: # lê e exibe conteúdo do arquivo
    print(f.read())

####**Conjuntos**
$P$: Conjunto dos produtos \\
$F$: Conjunto das fábricas

In [ ]:
P = range(4)
F = range(3)

####**Dados**

$D_{i}$: Demanda do produto $i$ \\
$L_{j}$: Capacidade da fábrica $j$ \\
$C_{ij}$: Custo de produção de uma unidade do produto $i$ na fábrica $j$

In [ ]:
C = [
    [41, 40, 37],
    [27, 29, 30],
    [28, 999, 27],
    [24, 23, 21]
]

L = [75,75,45]

D = [20,30,30,40]

In [ ]:
# matriz de aptidões
fit = [[41, 27, 28, 24],  # F1-P1, F1-P2, F1-P3, F1-P4
       [40,	29,	0,	23],  # F2-P1, F2-P2, F2-P3, F2-P4
       [37,	30,	27,	21]]  # F3-P1, F3-P2, F3-P3, F3-P4

n_ind = 3 # número de fábricas
n_prod = 4 # número de produtos

# Instanciando o modelo e atribuindo que a função objetivo é de minimização
model_1 = Model(sense=MINIMIZE, solver_name=CBC) # ... criar modelo

# Fazendo list comprehension para criar matriz de variaveis com a mesma
# estrutura do fit acima e iterando para representar 3 fab. e 4 prod.
x = [[model_1.add_var(var_type="INTEGER", name = f"x_{i+1}_{j+1}") for j in range(n_prod)]
            for i in range(n_ind)]

# Função objetivo
model_1.objective = fit[0][0]*x[0][0] + fit[0][1]*x[0][1] + fit[0][2]*x[0][2] + fit[0][3]*x[0][3] + \
                    fit[1][0]*x[1][0] + fit[1][1]*x[1][1] + fit[1][2]*x[1][2] + fit[1][3]*x[1][3] + \
                    fit[2][0]*x[2][0] + fit[2][1]*x[2][1] + fit[2][2]*x[2][2] + fit[2][3]*x[2][3]

# Restrições

# Fábricas
model_1 += x[0][0] + x[0][1] + x[0][2] + x[0][3] <= 75
model_1 += x[1][0] + x[1][1] + x[1][2] + x[1][3] <= 75
model_1 += x[2][0] + x[2][1] + x[2][2] + x[2][3] <= 45

# Produtos
model_1 += x[0][0] + x[1][0] + x[2][0] == 20
model_1 += x[0][1] + x[1][1] + x[2][1] == 30
model_1 += x[0][2] + x[1][2] + x[2][2] == 30
model_1 += x[0][3] + x[1][3] + x[2][3] == 40

model_1 += x[1][2] == 0

save(model_1, "model1.lp")

\Problem name: 

Minimize
OBJROW: 41 x_1_1 + 27 x_1_2 + 28 x_1_3 + 24 x_1_4 + 40 x_2_1 + 29 x_2_2 + 23 x_2_4 + 37 x_3_1 + 30 x_3_2 + 27 x_3_3
 + 21 x_3_4
Subject To
constr(0):  x_1_1 + x_1_2 + x_1_3 + x_1_4 <= 75
constr(1):  x_2_1 + x_2_2 + x_2_3 + x_2_4 <= 75
constr(2):  x_3_1 + x_3_2 + x_3_3 + x_3_4 <= 45
constr(3):  x_1_1 + x_2_1 + x_3_1 = 20
constr(4):  x_1_2 + x_2_2 + x_3_2 = 30
constr(5):  x_1_3 + x_2_3 + x_3_3 = 30
constr(6):  x_1_4 + x_2_4 + x_3_4 = 40
constr(7):  x_2_3 = 0
Bounds
End



In [ ]:
solve(model_1)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 3260.00

Solution:
x_1_1 = 0.00
x_1_2 = 30.00
x_1_3 = 30.00
x_1_4 = 0.00
x_2_1 = 0.00
x_2_2 = 0.00
x_2_3 = 0.00
x_2_4 = 15.00
x_3_1 = 20.00
x_3_2 = 0.00
x_3_3 = 0.00
x_3_4 = 25.00


## Exercício 2

Refaça o modelo anterior, dessa vez impedindo a divisão de produtos entre fábricas. Ou seja, cada produto deve ser integralmente produzido em uma única fábrica, para eliminar custos ocultos associados à divisão da produção. A cada fábrica deve ser destinado pelo menos um produto.

### Resolução

In [ ]:
# matriz de aptidões
fit = [[820, 810, 840, 960],  # F1-P1, F1-P2, F1-P3, F1-P4
       [800, 870,	0,	920],  # F2-P1, F2-P2, F2-P3, F2-P4
       [740, 900,	810,	840]]  # F3-P1, F3-P2, F3-P3, F3-P4

n_ind = 3 # número de fábricas
n_prod = 4 # número de produtos

# Instanciando o modelo e atribuindo que a função objetivo é de minimização
model_2 = Model(sense=MINIMIZE, solver_name=CBC) # ... criar modelo

# Fazendo list comprehension para criar matriz de variaveis com a mesma
# estrutura do fit acima e iterando para representar 3 fab. e 4 prod.
x = [[model_2.add_var(var_type=BINARY, name = f"x_{i+1}_{j+1}") for j in range(n_prod)]
            for i in range(n_ind)]

# Função objetivo
model_2.objective = fit[0][0]*x[0][0] + fit[0][1]*x[0][1] + fit[0][2]*x[0][2] + fit[0][3]*x[0][3] + \
                    fit[1][0]*x[1][0] + fit[1][1]*x[1][1] + fit[1][2]*x[1][2] + fit[1][3]*x[1][3] + \
                    fit[2][0]*x[2][0] + fit[2][1]*x[2][1] + fit[2][2]*x[2][2] + fit[2][3]*x[2][3]

# Restrições

# Fábricas
model_2 += 20*x[0][0] + 30*x[0][1] + 30*x[0][2] + 40*x[0][3] <= 75
model_2 += 20*x[1][0] + 30*x[1][1] + 30*x[1][2] + 40*x[1][3] <= 75
model_2 += 20*x[2][0] + 30*x[2][1] + 30*x[2][2] + 40*x[2][3] <= 45

# Produção por fábrica
model_2 += x[0][0] + x[0][1] + x[0][2] + x[0][3] >= 1
model_2 += x[1][0] + x[1][1] + x[1][2] + x[1][3] >= 1
model_2 += x[2][0] + x[2][1] + x[2][2] + x[2][3] >= 1

# Produtos
model_2 += x[0][0] + x[1][0] + x[2][0] == 1
model_2 += x[0][1] + x[1][1] + x[2][1] == 1
model_2 += x[0][2] + x[1][2] + x[2][2] == 1
model_2 += x[0][3] + x[1][3] + x[2][3] == 1
model_2 += x[1][2] == 0

save(model_2, "model2.lp")

\Problem name: 

Minimize
OBJROW: 820 x_1_1 + 810 x_1_2 + 840 x_1_3 + 960 x_1_4 + 800 x_2_1 + 870 x_2_2 + 920 x_2_4 + 740 x_3_1 + 900 x_3_2 + 810 x_3_3
 + 840 x_3_4
Subject To
constr(0):  20 x_1_1 + 30 x_1_2 + 30 x_1_3 + 40 x_1_4 <= 75
constr(1):  20 x_2_1 + 30 x_2_2 + 30 x_2_3 + 40 x_2_4 <= 75
constr(2):  20 x_3_1 + 30 x_3_2 + 30 x_3_3 + 40 x_3_4 <= 45
constr(3):  x_1_1 + x_1_2 + x_1_3 + x_1_4 >= 1
constr(4):  x_2_1 + x_2_2 + x_2_3 + x_2_4 >= 1
constr(5):  x_3_1 + x_3_2 + x_3_3 + x_3_4 >= 1
constr(6):  x_1_1 + x_2_1 + x_3_1 = 1
constr(7):  x_1_2 + x_2_2 + x_3_2 = 1
constr(8):  x_1_3 + x_2_3 + x_3_3 = 1
constr(9):  x_1_4 + x_2_4 + x_3_4 = 1
constr(10):  x_2_3 = 0
Bounds
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_1_3 <= 1
 0 <= x_1_4 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1
 0 <= x_2_3 <= 1
 0 <= x_2_4 <= 1
 0 <= x_3_1 <= 1
 0 <= x_3_2 <= 1
 0 <= x_3_3 <= 1
 0 <= x_3_4 <= 1
Integers
x_1_1 x_1_2 x_1_3 x_1_4 x_2_1 x_2_2 x_2_3 x_2_4 x_3_1 x_3_2 
x_3_3 x_3_4 
End



In [ ]:
solve(model_2)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 3290.00

Solution:
x_1_1 = 0.00
x_1_2 = 1.00
x_1_3 = 1.00
x_1_4 = 0.00
x_2_1 = 1.00
x_2_2 = 0.00
x_2_3 = 0.00
x_2_4 = 0.00
x_3_1 = 0.00
x_3_2 = 0.00
x_3_3 = 0.00
x_3_4 = 1.00
